In [4]:
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import twitter_samples, stopwords
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize
from nltk import FreqDist, classify, NaiveBayesClassifier
import re, string, random
from tkinter import *

def onButtonClick():
    positive_tweets = twitter_samples.strings('positive_tweets.json')
    negative_tweets = twitter_samples.strings('negative_tweets.json')
    text = twitter_samples.strings('tweets.20150430-223406.json')

    positive_tweet_tokens = twitter_samples.tokenized('positive_tweets.json')
    negative_tweet_tokens = twitter_samples.tokenized('negative_tweets.json')

    positive_cleaned_tokens_list = []
    negative_cleaned_tokens_list = []
    
    stop_words = stopwords.words('english')

    for tokens in positive_tweet_tokens:
        positive_cleaned_tokens_list.append(remove_noise(tokens, stop_words))

    for tokens in negative_tweet_tokens:
        negative_cleaned_tokens_list.append(remove_noise(tokens, stop_words))

    all_pos_words = get_all_words(positive_cleaned_tokens_list)

    positive_tokens_for_model = get_tweets_for_model(positive_cleaned_tokens_list)
    negative_tokens_for_model = get_tweets_for_model(negative_cleaned_tokens_list)

    positive_dataset = [(tweet_dict, "Positive")
                         for tweet_dict in positive_tokens_for_model]

    negative_dataset = [(tweet_dict, "Negative")
                         for tweet_dict in negative_tokens_for_model]

    dataset = positive_dataset + negative_dataset
    random.shuffle(dataset)

    train_data = dataset[:7000]
    test_data = dataset[7000:]

    classifier = NaiveBayesClassifier.train(train_data)

    print("Accuracy is:", classify.accuracy(classifier, test_data))

    custom_tweet = myTextBox.get()
    custom_tokens = remove_noise(word_tokenize(custom_tweet))
    sentiment = str(classifier.classify(dict([token, True] for token in custom_tokens)))
    var1 = "Comment: " + custom_tweet + "\nSentiment: " + sentiment
    if sentiment == "Positive":
        myLabel2 = Label(root, text = var1, bg = 'Green')
    else:
        myLabel2 = Label(root, text = var1, bg = 'Red')
    myLabel2.pack()

def remove_noise(tweet_tokens, stop_words = ()):

    cleaned_tokens = []

    for token, tag in pos_tag(tweet_tokens):
        token = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+#]|[!*\(\),]|'\
                       '(?:%[0-9a-fA-F][0-9a-fA-F]))+','', token)
        token = re.sub("(@[A-Za-z0-9_]+)","", token)

        if tag.startswith("NN"):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'

        lemmatizer = WordNetLemmatizer()
        token = lemmatizer.lemmatize(token, pos)

        if len(token) > 0 and token not in string.punctuation and token.lower() not in stop_words:
            cleaned_tokens.append(token.lower())
    return cleaned_tokens

def get_all_words(cleaned_tokens_list):
    for tokens in cleaned_tokens_list:
        for token in tokens:
            yield token

def get_tweets_for_model(cleaned_tokens_list):
    for tweet_tokens in cleaned_tokens_list:
        yield dict([token, True] for token in tweet_tokens)

if __name__ == "__main__":
    root = Tk()
    root.title("NLP Project")
    root.iconbitmap("NLP.ico") # H:\\CS\\Study Materials\\My Notes\\Notes\\NLP\\Mini Project\\NLP MiniProject007 PPT's Materials\\NLP.ico
    root.configure(bg='black')
    root.geometry('500x500+450+200')
    
    myLabel0 = Label(root, text = "Enter tweet here: ", fg = "blue")
    myLabel0.pack()
    myTextBox = Entry(root, width = 50, borderwidth = 2, bg = '#0066ff', fg = '#ffffff')
    myTextBox.pack()
    myButton = Button(root, text = "Start Sentiment Analysis", command = onButtonClick, fg = "green")
    myButton.pack()
    myButton2 = Button(root, text = "Exit", command = root.destroy, fg = "red")
    myButton2.pack()
    
    root.mainloop()
    
# Reference: https://www.digitalocean.com/community/tutorials/how-to-perform-sentiment-analysis-in-python-3-using-the-natural-language-toolkit-nltk